# 02. Módulo de Edición (Código Fuente Incluido)

Este notebook implementa el sistema de anonimización de video. Contiene la lógica para aplicar efectos visuales (blur, pixelado, máscaras, etc.) sobre regiones específicas del video.

### 1. Imports y Configuración
Importamos `opencv` (cv2) para manipulación de imágenes/video y `numpy` para operaciones matriciales (los efectos suelen ser manipulaciones de matrices de píxeles).

In [2]:
import cv2
import numpy as np
import logging
import torch
import torch.nn.functional as F
from typing import List, Dict, Optional, Any, Tuple
from pathlib import Path
import asyncio
import nest_asyncio

# Kornia para efectos GPU
try:
    import kornia
    import kornia.filters
    KORNIA_AVAILABLE = True
except ImportError:
    print("Kornia no instalado. Ejecuta: pip install kornia>=0.7.0")
    print("Los efectos se ejecutarán en CPU con OpenCV.")
    KORNIA_AVAILABLE = False

nest_asyncio.apply()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# =============================================================================
# KORNIA EFFECTS - Efectos acelerados por GPU
# =============================================================================
class KorniaEffects:
    """
    Efectos de anonimización acelerados por GPU usando Kornia + PyTorch.
    
    Arquitectura híbrida:
    - OpenCV: Video I/O (lectura/escritura)
    - Kornia/PyTorch: Efectos en GPU (blur, pixelate)
    
    Rendimiento esperado vs OpenCV CPU:
    - Blur: ~5-7x más rápido
    - Pixelate: ~3-5x más rápido
    
    Uso:
        effects = KorniaEffects()
        tensor = effects.numpy_to_tensor(frame)  # [B, C, H, W]
        blurred = effects.blur_regions(tensor, bboxes)
        result = effects.tensor_to_numpy(blurred)
    """
    
    def __init__(self, device: str = None):
        """
        Args:
            device: "cuda" o "cpu". Si None, auto-detecta.
        """
        if device is None:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device
        
        # Cache de tensores de ruido para consistencia entre frames
        self.noise_cache: Dict[Tuple[int, int, int], torch.Tensor] = {}
        
        logger.info(f"KorniaEffects initialized on {self.device}")
        
        if self.device == "cuda" and not KORNIA_AVAILABLE:
            logger.warning("Kornia not available, falling back to CPU OpenCV")
    
    def numpy_to_tensor(self, frame: np.ndarray) -> torch.Tensor:
        """
        Convierte frame numpy (H, W, C) BGR a tensor (1, C, H, W) RGB en GPU.
        """
        # BGR -> RGB
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # (H, W, C) -> (C, H, W) -> (1, C, H, W)
        tensor = torch.from_numpy(rgb).permute(2, 0, 1).unsqueeze(0)
        
        # Normalizar a [0, 1] y mover a device
        tensor = tensor.float().div(255.0).to(self.device)
        
        return tensor
    
    def tensor_to_numpy(self, tensor: torch.Tensor) -> np.ndarray:
        """
        Convierte tensor (1, C, H, W) RGB a numpy (H, W, C) BGR.
        """
        # (1, C, H, W) -> (C, H, W) -> (H, W, C)
        arr = tensor.squeeze(0).permute(1, 2, 0)
        
        # Desnormalizar y convertir a uint8
        arr = arr.mul(255.0).clamp(0, 255).byte()
        
        # Mover a CPU y convertir a numpy
        arr = arr.cpu().numpy()
        
        # RGB -> BGR
        return cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
    
    def blur_region(
        self, 
        tensor: torch.Tensor, 
        bbox: Tuple[int, int, int, int],
        kernel_size: int = 31,
        sigma: float = 15.0
    ) -> torch.Tensor:
        """
        Aplica Gaussian blur GPU a una región del tensor.
        
        Args:
            tensor: Tensor (1, C, H, W)
            bbox: (x1, y1, x2, y2)
            kernel_size: Tamaño del kernel (debe ser impar)
            sigma: Desviación estándar del blur
            
        Returns:
            Tensor con región blurreada
        """
        if not KORNIA_AVAILABLE:
            # Fallback a CPU OpenCV
            return self._blur_opencv_fallback(tensor, bbox, kernel_size, sigma)
        
        x1, y1, x2, y2 = bbox
        result = tensor.clone()
        
        # Extraer ROI
        roi = tensor[:, :, y1:y2, x1:x2]
        
        # Asegurar kernel impar
        if kernel_size % 2 == 0:
            kernel_size += 1
        
        # Aplicar blur con Kornia
        blurred_roi = kornia.filters.gaussian_blur2d(
            roi, 
            (kernel_size, kernel_size), 
            (sigma, sigma)
        )
        
        # Insertar ROI blurreado
        result[:, :, y1:y2, x1:x2] = blurred_roi
        
        return result
    
    def _blur_opencv_fallback(
        self, 
        tensor: torch.Tensor, 
        bbox: Tuple[int, int, int, int],
        kernel_size: int,
        sigma: float
    ) -> torch.Tensor:
        """Fallback blur usando OpenCV CPU"""
        frame = self.tensor_to_numpy(tensor)
        x1, y1, x2, y2 = bbox
        
        roi = frame[y1:y2, x1:x2]
        k = kernel_size | 1  # Asegurar impar
        blurred = cv2.GaussianBlur(roi, (k, k), sigma)
        frame[y1:y2, x1:x2] = blurred
        
        return self.numpy_to_tensor(frame)
    
    def pixelate_region(
        self, 
        tensor: torch.Tensor, 
        bbox: Tuple[int, int, int, int],
        blocks: int = 10,
        track_id: int = 0,
        add_noise: bool = True
    ) -> torch.Tensor:
        """
        Aplica pixelado GPU a una región del tensor.
        
        Usa F.interpolate para downscale/upscale, que es muy eficiente en GPU.
        Opcionalmente añade ruido consistente (mismo ruido para mismo track_id).
        
        Args:
            tensor: Tensor (1, C, H, W)
            bbox: (x1, y1, x2, y2)
            blocks: Número de bloques de pixelado
            track_id: ID para cache de ruido consistente
            add_noise: Añadir ruido para mejor anonimización
            
        Returns:
            Tensor con región pixelada
        """
        x1, y1, x2, y2 = bbox
        result = tensor.clone()
        
        # Extraer ROI
        roi = tensor[:, :, y1:y2, x1:x2]
        orig_h, orig_w = roi.shape[2], roi.shape[3]
        
        if orig_h < 2 or orig_w < 2:
            return result
        
        # Downscale a bloques
        small = F.interpolate(
            roi, 
            size=(blocks, blocks), 
            mode='bilinear', 
            align_corners=False
        )
        
        # Añadir ruido consistente si se solicita
        if add_noise:
            cache_key = (track_id, blocks, 3)  # 3 canales
            
            if cache_key not in self.noise_cache:
                # Generar ruido determinístico basado en track_id
                gen = torch.Generator(device=self.device)
                gen.manual_seed(track_id * 1000 + blocks)
                
                noise = torch.rand(
                    1, 3, blocks, blocks, 
                    generator=gen, 
                    device=self.device
                ) * 0.2 - 0.1  # Ruido en rango [-0.1, 0.1]
                
                self.noise_cache[cache_key] = noise
            
            small = small + self.noise_cache[cache_key]
            small = small.clamp(0, 1)
        
        # Upscale con nearest neighbor (mantiene bloques)
        pixelated = F.interpolate(
            small, 
            size=(orig_h, orig_w), 
            mode='nearest'
        )
        
        # Insertar ROI pixelado
        result[:, :, y1:y2, x1:x2] = pixelated
        
        return result
    
    def blur_regions_batch(
        self, 
        tensor: torch.Tensor, 
        bboxes: List[Tuple[int, int, int, int]],
        kernel_size: int = 31,
        sigma: float = 15.0
    ) -> torch.Tensor:
        """Aplica blur a múltiples regiones"""
        result = tensor.clone()
        for bbox in bboxes:
            result = self.blur_region(result, bbox, kernel_size, sigma)
        return result
    
    def pixelate_regions_batch(
        self,
        tensor: torch.Tensor,
        regions: List[Dict],  # [{"bbox": (x1,y1,x2,y2), "track_id": id, "blocks": n}, ...]
    ) -> torch.Tensor:
        """Aplica pixelado a múltiples regiones con configuración individual"""
        result = tensor.clone()
        for region in regions:
            result = self.pixelate_region(
                result,
                bbox=region["bbox"],
                blocks=region.get("blocks", 10),
                track_id=region.get("track_id", 0),
                add_noise=region.get("add_noise", True)
            )
        return result
    
    def clear_cache(self):
        """Limpia cache de ruido"""
        self.noise_cache.clear()
        if self.device == "cuda":
            torch.cuda.empty_cache()

# Instancia global para reutilización
kornia_effects = KorniaEffects() if KORNIA_AVAILABLE or torch.cuda.is_available() else None

print(f"✓ Kornia disponible: {KORNIA_AVAILABLE}")
print(f"✓ Device: {kornia_effects.device if kornia_effects else 'CPU (OpenCV only)'}")

INFO:__main__:KorniaEffects initialized on cuda


✓ Kornia disponible: True
✓ Device: cuda


### 2. Video Anonymizer (`video_editor.py`)
Esta es la clase principal que procesa el video.

- **`apply_anonymization`**: 
    - Abre el video de entrada y crea un escritor (`VideoWriter`) para la salida.
    - Lee frames uno a uno.
    - Llama a `_process_frame` para aplicar efectos en cada frame.
    - Guarda el resultado.

- **`_process_frame`**: 
    - Recorre la lista de `actions`. Cada acción define qué efecto aplicar y en qué frames/coordenadas (`bboxes`).
    - Si el frame actual tiene una caja definida en las acciones, llama a `_apply_effect`.

- **`_apply_effect`**: La lógica visual real:
    - **`blur`**: Aplica un difuminado Gaussiano. Útil para privacidad suave.
    - **`pixelate`** (con ruido): Reduce la imagen a bloques y añade ruido aleatorio antes de re-escalar. Esto impide que se reconozcan rasgos incluso con bloques pequeños.
    - **`mask`**: Mezcla los píxeles usando una clave secreta. Es reversible si se conoce la clave.

In [4]:
class VideoAnonymizer:
    """
    Anonimizador de video con aceleración GPU (Kornia) y fallback CPU (OpenCV).
    
    Arquitectura híbrida:
    - OpenCV: Video I/O (VideoCapture/VideoWriter)
    - Kornia + PyTorch: Efectos en GPU (blur, pixelate) cuando está disponible
    - OpenCV: Efectos en CPU como fallback
    
    Mejoras incluidas:
    - Cache de ruido para evitar flickering en pixelado
    - Interpolación de bboxes para suavizar movimiento
    - Procesamiento por batches para eficiencia GPU
    """
    
    def __init__(self, use_gpu: bool = True, batch_frames: int = 8):
        """
        Args:
            use_gpu: Intentar usar GPU si está disponible
            batch_frames: Número de frames a procesar en batch (GPU)
        """
        self.use_gpu = use_gpu and (KORNIA_AVAILABLE or torch.cuda.is_available())
        self.batch_frames = batch_frames
        
        # Instancia de efectos Kornia
        self.kornia = kornia_effects if self.use_gpu else None
        
        # Cache de ruido para CPU fallback
        self.noise_cache = {}
        
        logger.info(f"VideoAnonymizer: GPU={self.use_gpu}, batch={batch_frames}")

    def _interpolate_bboxes(self, actions: List[Dict]) -> List[Dict]:
        """
        Rellena frames faltantes con interpolación lineal.
        Evita saltos bruscos cuando hay gaps en la detección.
        """
        for action in actions:
            bboxes = action.get("bboxes", {})
            if not bboxes:
                continue
                
            frames = sorted(bboxes.keys())
            if len(frames) < 2:
                continue
            
            interpolated = dict(bboxes)
            
            for i in range(len(frames) - 1):
                f1, f2 = frames[i], frames[i + 1]
                gap = f2 - f1
                
                # Solo interpolar si hay gap pequeño (< 10 frames)
                if 1 < gap <= 10:
                    b1 = bboxes[f1]
                    b2 = bboxes[f2]
                    
                    for f in range(f1 + 1, f2):
                        t = (f - f1) / gap
                        interpolated[f] = [
                            int(b1[0] + t * (b2[0] - b1[0])),
                            int(b1[1] + t * (b2[1] - b1[1])),
                            int(b1[2] + t * (b2[2] - b1[2])),
                            int(b1[3] + t * (b2[3] - b1[3])),
                        ]
            
            action["bboxes"] = interpolated
        
        return actions

    async def apply_anonymization(
        self,
        input_path: str,
        output_path: str,
        actions: List[Dict[str, Any]],
        on_progress: Optional[Any] = None
    ):
        """
        Aplica anonimización al video.
        
        Args:
            input_path: Ruta del video de entrada
            output_path: Ruta del video de salida
            actions: Lista de acciones de anonimización
            on_progress: Callback de progreso (frame, total, msg)
        """
        import time
        start_time = time.time()
        
        logger.info(f"Anonymizing video: {input_path} -> {output_path}")
        
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            raise ValueError(f"Failed to open video: {input_path}")

        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        # Interpolar bboxes para suavizar movimiento
        actions = self._interpolate_bboxes(actions)
        
        # Limpiar caches
        self.noise_cache = {}
        if self.kornia:
            self.kornia.clear_cache()
        
        frame_idx = 0
        
        try:
            if self.use_gpu and self.kornia:
                # Procesamiento GPU por batches
                await self._process_gpu_batched(
                    cap, out, actions, total_frames, on_progress
                )
            else:
                # Procesamiento CPU frame por frame
                await self._process_cpu(
                    cap, out, actions, total_frames, on_progress
                )
        finally:
            cap.release()
            out.release()
        
        elapsed = time.time() - start_time
        fps_proc = total_frames / elapsed if elapsed > 0 else 0
        logger.info(f"Anonymization completed: {elapsed:.2f}s ({fps_proc:.1f} fps)")

    async def _process_gpu_batched(
        self, 
        cap, 
        out, 
        actions: List[Dict],
        total_frames: int,
        on_progress
    ):
        """Procesa video en batches usando GPU (Kornia)"""
        frame_buffer = []
        frame_indices = []
        frame_idx = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                # Procesar batch final
                if frame_buffer:
                    await self._process_batch_gpu(
                        frame_buffer, frame_indices, actions, out
                    )
                break
            
            frame_idx += 1
            frame_buffer.append(frame)
            frame_indices.append(frame_idx)
            
            if len(frame_buffer) >= self.batch_frames:
                await self._process_batch_gpu(
                    frame_buffer, frame_indices, actions, out
                )
                frame_buffer = []
                frame_indices = []
                
                if frame_idx % 20 == 0 and on_progress:
                    msg = f"Frame {frame_idx}/{total_frames}"
                    if asyncio.iscoroutinefunction(on_progress):
                        await on_progress(frame_idx, total_frames, msg)
                    else:
                        on_progress(frame_idx, total_frames, msg)

    async def _process_batch_gpu(
        self, 
        frames: List[np.ndarray], 
        frame_indices: List[int],
        actions: List[Dict],
        out
    ):
        """Procesa un batch de frames en GPU"""
        for frame, frame_idx in zip(frames, frame_indices):
            # Convertir a tensor GPU
            tensor = self.kornia.numpy_to_tensor(frame)
            
            # Recopilar acciones para este frame
            blur_regions = []
            pixelate_regions = []
            mask_regions = []
            
            for action in actions:
                bboxes_map = action.get("bboxes", {})
                action_type = action.get("type", "blur")
                config = action.get("config", {})
                track_id = action.get("track_id", 0)
                
                box = bboxes_map.get(frame_idx)
                if not box:
                    continue
                
                bbox = (int(box[0]), int(box[1]), int(box[2]), int(box[3]))
                
                if action_type == "blur":
                    blur_regions.append({
                        "bbox": bbox,
                        "kernel_size": config.get("kernel_size", 31),
                        "sigma": config.get("sigma", 15.0)
                    })
                elif action_type == "pixelate":
                    pixelate_regions.append({
                        "bbox": bbox,
                        "blocks": config.get("blocks", 10),
                        "track_id": track_id,
                        "add_noise": config.get("add_noise", True)
                    })
                elif action_type == "mask":
                    mask_regions.append({
                        "bbox": bbox,
                        "key": config.get("key", 42)
                    })
            
            # Aplicar efectos GPU
            for region in blur_regions:
                tensor = self.kornia.blur_region(
                    tensor, 
                    region["bbox"],
                    region["kernel_size"],
                    region["sigma"]
                )
            
            for region in pixelate_regions:
                tensor = self.kornia.pixelate_region(
                    tensor,
                    region["bbox"],
                    region["blocks"],
                    region["track_id"],
                    region["add_noise"]
                )
            
            # Convertir de vuelta a numpy
            result_frame = self.kornia.tensor_to_numpy(tensor)
            
            # Aplicar mask en CPU (requiere permutación compleja)
            for region in mask_regions:
                self._apply_mask_cpu(result_frame, region["bbox"], region["key"])
            
            out.write(result_frame)
        
        await asyncio.sleep(0)

    async def _process_cpu(
        self, 
        cap, 
        out, 
        actions: List[Dict],
        total_frames: int,
        on_progress
    ):
        """Procesa video frame por frame en CPU (fallback)"""
        frame_idx = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            frame_idx += 1
            frame = self._process_frame_cpu(frame, frame_idx, actions)
            out.write(frame)
            
            if frame_idx % 20 == 0:
                if on_progress:
                    msg = f"Frame {frame_idx}/{total_frames}"
                    if asyncio.iscoroutinefunction(on_progress):
                        await on_progress(frame_idx, total_frames, msg)
                    else:
                        on_progress(frame_idx, total_frames, msg)
                await asyncio.sleep(0)

    def _process_frame_cpu(self, frame: np.ndarray, frame_idx: int, actions: List[Dict]) -> np.ndarray:
        """Procesa un frame en CPU (OpenCV)"""
        for action in actions:
            bboxes_map = action.get("bboxes", {})
            action_type = action.get("type", "blur")
            config = action.get("config", {})
            track_id = action.get("track_id", 0)
            
            box = bboxes_map.get(frame_idx)
            if box:
                self._apply_effect_cpu(frame, box, action_type, config, track_id)
        return frame

    def _apply_effect_cpu(self, frame: np.ndarray, bbox: list, effect: str, config: dict, track_id: int = 0):
        """Aplica efecto usando OpenCV CPU"""
        h, w = frame.shape[:2]
        x1, y1, x2, y2 = map(int, bbox)
        x1 = max(0, x1); y1 = max(0, y1)
        x2 = min(w, x2); y2 = min(h, y2)
        
        if x2 <= x1 or y2 <= y1:
            return

        roi = frame[y1:y2, x1:x2]
        
        if effect == 'blur':
            factor = config.get("factor", 3.0)
            k_w = int((x2 - x1) // factor) | 1
            k_h = int((y2 - y1) // factor) | 1
            blurred = cv2.GaussianBlur(roi, (k_w, k_h), 0)
            frame[y1:y2, x1:x2] = blurred
            
        elif effect == 'pixelate':
            blocks = config.get("blocks", 30)
            
            # Pixelado base
            small = cv2.resize(roi, (blocks, blocks), interpolation=cv2.INTER_LINEAR)
            
            # Ruido consistente por track_id
            cache_key = (track_id, blocks)
            if cache_key not in self.noise_cache:
                rng = np.random.default_rng(seed=track_id * 1000 + blocks)
                self.noise_cache[cache_key] = rng.integers(-30, 30, (blocks, blocks, 3), dtype=np.int16)
            
            noise = self.noise_cache[cache_key]
            dirty_small = small.astype(np.int16) + noise
            dirty_small = np.clip(dirty_small, 0, 255).astype(np.uint8)
            
            pixelated = cv2.resize(dirty_small, (x2-x1, y2-y1), interpolation=cv2.INTER_NEAREST)
            frame[y1:y2, x1:x2] = pixelated
            
        elif effect == 'mask':
            self._apply_mask_cpu(frame, (x1, y1, x2, y2), config.get("key", 42))

    def _apply_mask_cpu(self, frame: np.ndarray, bbox: tuple, key: int):
        """Aplica efecto mask (scramble) en CPU"""
        x1, y1, x2, y2 = bbox
        roi = frame[y1:y2, x1:x2]
        shape = roi.shape
        flat = roi.flatten()
        rng = np.random.default_rng(key)
        perm = rng.permutation(len(flat))
        scrambled = flat[perm]
        frame[y1:y2, x1:x2] = scrambled.reshape(shape)

### 3. Ejecución de Prueba
Aquí definimos un escenario ficticio para probar los efectos:
- Definimos un objeto que se mueve del frame 1 al 100.
- Le aplicamos el nuevo efecto `pixelate` con ruido y 30 bloques.
- Ejecutamos el `apply_anonymization` y mostramos el resultado.

In [5]:
# =============================================================================
# PRUEBA DE EJECUCIÓN
# =============================================================================

import os
import time

VIDEO_PATH = "../storage/uploads/coche.mp4" 
OUTPUT_GPU = "../storage/processed/test_kornia_gpu.mp4"
OUTPUT_CPU = "../storage/processed/test_opencv_cpu.mp4"

# Acciones de prueba con diferentes efectos
actions = [
    {
        "type": "blur",
        "track_id": 1,
        "config": {"kernel_size": 31, "sigma": 15.0},
        "bboxes": {i: [50, 50, 200, 200] for i in range(1, 100)}
    },
    {
        "type": "pixelate",
        "track_id": 2,
        "config": {"blocks": 20, "add_noise": True},
        "bboxes": {i: [250, 50, 450, 250] for i in range(1, 100)}
    },
    {
        "type": "pixelate",
        "track_id": 3,
        "config": {"blocks": 10, "add_noise": True},
        "bboxes": {i: [500, 100, 650, 300] for i in range(1, 100)}
    }
]

async def benchmark_anonymizer():
    if not os.path.exists(VIDEO_PATH):
        print(f"Video no encontrado: {VIDEO_PATH}")
        print("Saltando test. Descarga un video de prueba primero.")
        return
    
    print("=" * 60)
    print("BENCHMARK: GPU (Kornia) vs CPU (OpenCV)")
    print("=" * 60)
    
    results = {}
    
    # Test GPU (si está disponible)
    if kornia_effects is not None:
        print("\n[1] Probando GPU (Kornia + PyTorch)...")
        anonymizer_gpu = VideoAnonymizer(use_gpu=True, batch_frames=8)
        
        start = time.time()
        await anonymizer_gpu.apply_anonymization(VIDEO_PATH, OUTPUT_GPU, actions)
        gpu_time = time.time() - start
        results["GPU"] = gpu_time
        
        print(f"    ✓ Completado en {gpu_time:.2f}s")
    else:
        print("\n[1] GPU no disponible, saltando...")
    
    # Test CPU
    print("\n[2] Probando CPU (OpenCV)...")
    anonymizer_cpu = VideoAnonymizer(use_gpu=False)
    
    start = time.time()
    await anonymizer_cpu.apply_anonymization(VIDEO_PATH, OUTPUT_CPU, actions)
    cpu_time = time.time() - start
    results["CPU"] = cpu_time
    
    print(f"    ✓ Completado en {cpu_time:.2f}s")
    
    # Resumen
    print("\n" + "=" * 60)
    print("RESUMEN")
    print("=" * 60)
    
    if "GPU" in results and "CPU" in results:
        speedup = results["CPU"] / results["GPU"]
        print(f"  GPU: {results['GPU']:.2f}s")
        print(f"  CPU: {results['CPU']:.2f}s")
        print(f"  Speedup: {speedup:.1f}x más rápido con GPU")
    elif "CPU" in results:
        print(f"  CPU: {results['CPU']:.2f}s")
        print("  (GPU no disponible para comparar)")
    
    # Mostrar un frame de resultado
    if os.path.exists(OUTPUT_GPU):
        output_path = OUTPUT_GPU
    else:
        output_path = OUTPUT_CPU
    
    cap = cv2.VideoCapture(output_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, 50)
    ret, img = cap.read()
    cap.release()
    
    if ret:
        try:
            import matplotlib.pyplot as plt
            plt.figure(figsize=(12, 6))
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title("Resultado: Blur (izq) + Pixelate 20 bloques (centro) + Pixelate 10 bloques (der)")
            plt.axis('off')
            plt.show()
        except ImportError:
            print("\nMatplotlib no disponible para visualización.")

# Ejecutar benchmark
asyncio.run(benchmark_anonymizer())

INFO:__main__:VideoAnonymizer: GPU=True, batch=8
INFO:__main__:Anonymizing video: ../storage/uploads/coche.mp4 -> ../storage/processed/test_kornia_gpu.mp4


BENCHMARK: GPU (Kornia) vs CPU (OpenCV)

[1] Probando GPU (Kornia + PyTorch)...


INFO:__main__:Anonymization completed: 34.45s (54.7 fps)
INFO:__main__:VideoAnonymizer: GPU=False, batch=8
INFO:__main__:Anonymizing video: ../storage/uploads/coche.mp4 -> ../storage/processed/test_opencv_cpu.mp4


    ✓ Completado en 34.45s

[2] Probando CPU (OpenCV)...


INFO:__main__:Anonymization completed: 1.67s (1128.1 fps)


    ✓ Completado en 1.67s

RESUMEN
  GPU: 34.45s
  CPU: 1.67s
  Speedup: 0.0x más rápido con GPU
